# eDreams test for Data Scientist Role

*__Autor:__ Tomas Ruan Rollan*
<br>
*__LinkedIn:__ https://www.linkedin.com/in/tomas-ruan/*
<br>
*__Email:__ tomruarol@gmail.com*

# Index

1. [Imports](#1)   

2. [EDA](#2) <br>
    3.1 [Duplicate Check](#2.1)    
    3.2 [Target Variable Transformation](#2.2)    
    3.3 [Validation Strategy](#2.3)    
    3.4 [Data Cleaning & Transformation](#2.4)   
    3.5 [Dates conversion](#2.5)    
    3.6 [Label Encoding / One Hot Encoding](#2.6)       
    3.7 [Pickup Locations](#2.7)
     
3. [Feature Engineering](#3)    
    3.1 [Timing Features](#3.1)    
    
4. [Oversampling](#4)

5. [Modeling](#5)    
    5.1 [Setting K folds](#5.1)        
    5.2 [Parameter tunning](#5.2)       
    5.4 [Get the id column](#5.3)  

6. [Submission](#6) 

<a id="1"></a>
### Imports

In [3]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.metrics import accuracy_score

# For oversampling
from imblearn.over_sampling import SMOTE;

# Figures inline and set visualization style
%matplotlib inline
sns.set() #Different type of visualization

# import the necessary modelling algorithms

# Classifiaction
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Model selection
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder,PolynomialFeatures

# Evaluation metrics
from sklearn.metrics import accuracy_score,precision_score,f1_score

# Show multiple statements at once
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<a id="2"></a>
### EDA

In [16]:
# We import the data
df_train = pd.read_csv('input/train.csv', sep = ';')
df_test = pd.read_csv('input/test.csv', sep = ';')

# We take a look at the dataset to see that data was imported correctly
df_train.head()
df_test.head()

,ID,TIMESTAMP,WEBSITE,GDS,DEPARTURE,ARRIVAL,ADULTS,CHILDREN,INFANTS,TRAIN,HAUL_TYPE,DISTANCE,DEVICE,TRIP_TYPE,PRODUCT,SMS,EXTRA_BAGGAGE,NO_GDS
0,0,01/July,EDES,1,22/July,25/July,1,0,0,False,DOMESTIC,"628,844",TABLET,ROUND_TRIP,TRIP,True,False,0
1,1,01/July,EDIT,0,29/July,29/July,1,0,0,False,CONTINENTAL,"1281,43",SMARTPHONE,ONE_WAY,TRIP,False,False,1
2,2,01/July,OPUK,2,29/July,19/August,1,0,0,False,CONTINENTAL,"1730,35",TABLET,ROUND_TRIP,TRIP,True,False,0
3,3,01/July,OPIT,0,24/July,04/August,1,0,0,False,DOMESTIC,"652,702",SMARTPHONE,MULTI_DESTINATION,TRIP,False,False,2
4,4,01/July,EDES,0,11/August,11/August,1,0,0,False,CONTINENTAL,"1717,85",COMPUTER,ONE_WAY,TRIP,False,False,1


,ID,TIMESTAMP,WEBSITE,GDS,DEPARTURE,ARRIVAL,ADULTS,CHILDREN,INFANTS,TRAIN,HAUL_TYPE,DISTANCE,DEVICE,TRIP_TYPE,PRODUCT,SMS,NO_GDS
0,0,03/July,EDES,1,22/July,22/July,1,0,0,False,DOMESTIC,"3425,95",COMPUTER,ONE_WAY,TRIP,True,0
1,1,03/July,GOFR,1,05/July,22/August,1,0,0,False,INTERCONTINENTAL,"3206,92",SMARTPHONE,ROUND_TRIP,TRIP,False,0
2,2,03/July,OPGB,1,22/July,12/August,1,0,0,False,INTERCONTINENTAL,"6605,22",COMPUTER,ROUND_TRIP,TRIP,False,0
3,3,03/July,EDUK,0,06/July,08/July,1,0,0,False,CONTINENTAL,"1302,29",SMARTPHONE,ROUND_TRIP,TRIP,False,2
4,4,03/July,OPDE,1,10/September,10/September,1,0,0,False,INTERCONTINENTAL,"6770,78",COMPUTER,ONE_WAY,TRIP,True,0


In [17]:
df_train.tail()

,ID,TIMESTAMP,WEBSITE,GDS,DEPARTURE,ARRIVAL,ADULTS,CHILDREN,INFANTS,TRAIN,HAUL_TYPE,DISTANCE,DEVICE,TRIP_TYPE,PRODUCT,SMS,EXTRA_BAGGAGE,NO_GDS
49995,49995,02/July,EDUK,1,02/July,02/July,2,0,0,False,CONTINENTAL,"1035,13",COMPUTER,ONE_WAY,TRIP,True,True,0
49996,49996,02/July,EDPT,1,11/August,19/August,2,1,0,False,CONTINENTAL,"1152,2",SMARTPHONE,ROUND_TRIP,TRIP,True,False,0
49997,49997,02/July,GOFR,1,09/September,23/September,1,0,0,False,INTERCONTINENTAL,"2236,54",SMARTPHONE,ROUND_TRIP,TRIP,False,False,0
49998,49998,02/July,EDPT,2,05/July,21/July,1,0,0,False,CONTINENTAL,"1312,48",COMPUTER,ROUND_TRIP,TRIP,False,True,0
49999,49999,02/July,GOFR,0,06/July,06/July,1,0,0,False,INTERCONTINENTAL,"2403,31",SMARTPHONE,ONE_WAY,TRIP,False,False,1


In [18]:
# Let's check the number of rows and columns of train and test
print('We have {} training rows and {} test rows.'.format(df_train.shape[0], df_test.shape[0]))
print('We have {} training columns and {} test columns.'.format(len(df_train.columns), len(df_test.columns))) 

We have 50000 training rows and 30000 test rows.
We have 18 training columns and 17 test columns.


In [19]:
# We take a first look at the dataset
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 18 columns):
ID               50000 non-null int64
TIMESTAMP        50000 non-null object
WEBSITE          50000 non-null object
GDS              50000 non-null int64
DEPARTURE        50000 non-null object
ARRIVAL          50000 non-null object
ADULTS           50000 non-null int64
CHILDREN         50000 non-null int64
INFANTS          50000 non-null int64
TRAIN            50000 non-null bool
HAUL_TYPE        50000 non-null object
DISTANCE         50000 non-null object
DEVICE           49867 non-null object
TRIP_TYPE        50000 non-null object
PRODUCT          50000 non-null object
SMS              50000 non-null bool
EXTRA_BAGGAGE    50000 non-null bool
NO_GDS           50000 non-null int64
dtypes: bool(3), int64(6), object(9)
memory usage: 5.9+ MB


In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 17 columns):
ID           30000 non-null int64
TIMESTAMP    30000 non-null object
WEBSITE      30000 non-null object
GDS          30000 non-null int64
DEPARTURE    30000 non-null object
ARRIVAL      30000 non-null object
ADULTS       30000 non-null int64
CHILDREN     30000 non-null int64
INFANTS      30000 non-null int64
TRAIN        30000 non-null bool
HAUL_TYPE    30000 non-null object
DISTANCE     30000 non-null object
DEVICE       29948 non-null object
TRIP_TYPE    30000 non-null object
PRODUCT      30000 non-null object
SMS          30000 non-null bool
NO_GDS       30000 non-null int64
dtypes: bool(2), int64(6), object(9)
memory usage: 3.5+ MB


We can see we have some missing values in the `DEVICE` column in the train dataset. Later on we will decide what to do with them. <br>

Regarding the `ID` column. It tells us the same as the index from our dataframe, hence it's not giving us any extra information and because of all the entries are independent between each other we can safely drop it.

In [21]:
# We drop the ID column both in the train & test datset
df_train = df_train.drop('ID',axis=1)
df_test = df_test.drop('ID',axis=1)

In [22]:
df_train.head()
df_test.head()

,TIMESTAMP,WEBSITE,GDS,DEPARTURE,ARRIVAL,ADULTS,CHILDREN,INFANTS,TRAIN,HAUL_TYPE,DISTANCE,DEVICE,TRIP_TYPE,PRODUCT,SMS,EXTRA_BAGGAGE,NO_GDS
0,01/July,EDES,1,22/July,25/July,1,0,0,False,DOMESTIC,"628,844",TABLET,ROUND_TRIP,TRIP,True,False,0
1,01/July,EDIT,0,29/July,29/July,1,0,0,False,CONTINENTAL,"1281,43",SMARTPHONE,ONE_WAY,TRIP,False,False,1
2,01/July,OPUK,2,29/July,19/August,1,0,0,False,CONTINENTAL,"1730,35",TABLET,ROUND_TRIP,TRIP,True,False,0
3,01/July,OPIT,0,24/July,04/August,1,0,0,False,DOMESTIC,"652,702",SMARTPHONE,MULTI_DESTINATION,TRIP,False,False,2
4,01/July,EDES,0,11/August,11/August,1,0,0,False,CONTINENTAL,"1717,85",COMPUTER,ONE_WAY,TRIP,False,False,1


,TIMESTAMP,WEBSITE,GDS,DEPARTURE,ARRIVAL,ADULTS,CHILDREN,INFANTS,TRAIN,HAUL_TYPE,DISTANCE,DEVICE,TRIP_TYPE,PRODUCT,SMS,NO_GDS
0,03/July,EDES,1,22/July,22/July,1,0,0,False,DOMESTIC,"3425,95",COMPUTER,ONE_WAY,TRIP,True,0
1,03/July,GOFR,1,05/July,22/August,1,0,0,False,INTERCONTINENTAL,"3206,92",SMARTPHONE,ROUND_TRIP,TRIP,False,0
2,03/July,OPGB,1,22/July,12/August,1,0,0,False,INTERCONTINENTAL,"6605,22",COMPUTER,ROUND_TRIP,TRIP,False,0
3,03/July,EDUK,0,06/July,08/July,1,0,0,False,CONTINENTAL,"1302,29",SMARTPHONE,ROUND_TRIP,TRIP,False,2
4,03/July,OPDE,1,10/September,10/September,1,0,0,False,INTERCONTINENTAL,"6770,78",COMPUTER,ONE_WAY,TRIP,True,0


<a id="2.1"></a>
###### Duplicate Check

Let's check if there are any duplicates.

In [23]:
# We check if we have duplicates
df_train.duplicated().any()
df_test.duplicated().any()

True

True

It looks like we have no duplicates on the dataset, so one problem less!

<a id="2.1"></a>
###### Dates Conversion

As we saw before, our dates were `object` type. We will convert them into `datetime` type. <br>
Here we find a problem. <br>
Our dates have a day and a month respectively `01/July` for example, while the year is missing. This leads to a problem when trying to convert the dates as it will introduce a year by its own (year 1900). Hence, we have to come up with something to treat our dates and make them useful: <br>

- Are all dates in the same year?

    If they were not from the same year we could do some comparatives based on months of different years, or seasons,     for example, but as we do not know if they are from the same year we can't do this type of analysis.
    If they were from the same year we could just take into account months and days and do the analysis based only on     those two.
    
    
- Assuming they are all from the same year

    The easiest and the one which makes more sense here is to assume all of them are from the same year (1900    example, as it filled by default with `to_datetime` method from `pandas` library), just to fill the year in the format, so we are able to convert them to datetime format and then proceed with the analysis of this dates taking into account only the day and the month, forgetting about the year.

Before doing all this, there are some dates that do not have the same format as the rest. <br>
While most the dates have dd/M format (04/July), there are some that have dd-m format (17-may). This leads to problems when converting the dates. 

Let's replace the`-` symbols with `/` so all the dates have the same format.

In [26]:
# We replace the symbols in both datasets and in all the columns dates just in case

# train dataset
df_train['TIMESTAMP'] = df_train['TIMESTAMP'].str.replace('-', '/')
df_train['DEPARTURE'] = df_train['DEPARTURE'].str.replace('-', '/')
df_train['ARRIVAL'] = df_train['ARRIVAL'].str.replace('-', '/')

#test dataset
df_test['TIMESTAMP'] = asa['TIMESTAMP'].str.replace('-', '/')
df_test['DEPARTURE'] = asa['DEPARTURE'].str.replace('-', '/')
df_test['ARRIVAL'] = asa['ARRIVAL'].str.replace('-', '/')

In [27]:
# Let's look at 1 register
x = df_train['TIMESTAMP'][0]
x

'01/July'

In [28]:
# We add the year to the dates columns

# for the train dataset
df_train['TIMESTAMP'] = pd.to_datetime(df_train['TIMESTAMP'], format='%d/%B')
df_train['DEPARTURE'] = pd.to_datetime(df_train['DEPARTURE'], format='%d/%B')
df_train['ARRIVAL'] = pd.to_datetime(df_train['ARRIVAL'], format='%d/%B')

#for the test dataset
df_test['TIMESTAMP'] = pd.to_datetime(df_test['TIMESTAMP'], format='%d/%B')
df_test['DEPARTURE'] = pd.to_datetime(df_test['DEPARTURE'], format='%d/%B')
df_test['ARRIVAL'] = pd.to_datetime(df_test['ARRIVAL'], format='%d/%B')

# We add the year to the dates columns

# for the train dataset
df_train['TIMESTAMP'] = df_train['TIMESTAMP'].apply(lambda x: str(x) + '/2019')
df_train['DEPARTURE'] = df_train['DEPARTURE'].apply(lambda x: str(x) + '/2019')
df_train['ARRIVAL'] = df_train['ARRIVAL'].apply(lambda x: str(x) + '/2019')

#for the test dataset
df_test['TIMESTAMP'] = df_test['TIMESTAMP'].apply(lambda x: str(x) + '/2019')
df_test['DEPARTURE'] = df_test['DEPARTURE'].apply(lambda x: str(x) + '/2019')
df_test['ARRIVAL'] = df_test['ARRIVAL'].apply(lambda x: str(x) + '/2019')

In [29]:
# We take a look to see that it has been done properly
x = df_train['DEPARTURE'][2]
y = df_test['ARRIVAL'][2]
x
y

Timestamp('1900-07-29 00:00:00')

Timestamp('1900-08-19 00:00:00')

In [30]:
df_train.dtypes

TIMESTAMP        datetime64[ns]
WEBSITE                  object
GDS                       int64
DEPARTURE        datetime64[ns]
ARRIVAL          datetime64[ns]
ADULTS                    int64
CHILDREN                  int64
INFANTS                   int64
TRAIN                      bool
HAUL_TYPE                object
DISTANCE                 object
DEVICE                   object
TRIP_TYPE                object
PRODUCT                  object
SMS                        bool
EXTRA_BAGGAGE              bool
NO_GDS                    int64
dtype: object

As we can see our dates are `datetime` type, so we can now proceed with the analysis and use them.

In [ ]:
gg = pd.to_datetime(df_train['TIMESTAMP'], format='%d/%B/%y')

In [ ]:
df_train['TIMESTAMP'] = df_train['TIMESTAMP'].apply(lambda x: x.replace('-', '/'))

In [ ]:
#df_train['TIMESTAMP'] = pd.to_datetime(x.strip(), format='%d/%b')

df_train['TIMESTAMP'] = pd.datetime.strptime('01-07', '%d-%m')

In [ ]:
x = pd.to_datetime('01-07', format='%d-%m')
x

<a id="3.6"></a>
#### Label Encoding / One Hot Encoding

Most of the features in our dataset are categorical, for example: `WEBSITE` has values like `EDES, GOFT, EDUK, ...` and `DEVICE` has values such as `COMPUTER, SMARTPHONE, ...` for instance. <br>
We need to label this data in order to assign a number to each category, this is converting our categorical data into numerical data so our model can handle it.

In [ ]:
# Encogind for the train dataset
for column in df_train.columns:
    if df_train[column].dtype == type(object):
        le = LabelEncoder()
        df_train[column] = le.fit_transform(df_train[column])
        
# Encoding for the test dataset        
for column in df_test.columns:
    if df_test[column].dtype == type(object):
        le = LabelEncoder()
        df_test[column] = le.fit_transform(df_test[column])

In [4]:
asa = pd.read_csv('input/train.csv', sep = ';')

In [5]:
asa.head()

,ID,TIMESTAMP,WEBSITE,GDS,DEPARTURE,ARRIVAL,ADULTS,CHILDREN,INFANTS,TRAIN,HAUL_TYPE,DISTANCE,DEVICE,TRIP_TYPE,PRODUCT,SMS,EXTRA_BAGGAGE,NO_GDS
0,0,01/July,EDES,1,22/July,25/July,1,0,0,False,DOMESTIC,"628,844",TABLET,ROUND_TRIP,TRIP,True,False,0
1,1,01/July,EDIT,0,29/July,29/July,1,0,0,False,CONTINENTAL,"1281,43",SMARTPHONE,ONE_WAY,TRIP,False,False,1
2,2,01/July,OPUK,2,29/July,19/August,1,0,0,False,CONTINENTAL,"1730,35",TABLET,ROUND_TRIP,TRIP,True,False,0
3,3,01/July,OPIT,0,24/July,04/August,1,0,0,False,DOMESTIC,"652,702",SMARTPHONE,MULTI_DESTINATION,TRIP,False,False,2
4,4,01/July,EDES,0,11/August,11/August,1,0,0,False,CONTINENTAL,"1717,85",COMPUTER,ONE_WAY,TRIP,False,False,1


In [6]:
gh = pd.to_datetime(asa['TIMESTAMP'], format='%d/%B')

In [7]:
gh

0       1900-07-01
1       1900-07-01
2       1900-07-01
3       1900-07-01
4       1900-07-01
           ...    
49995   1900-07-02
49996   1900-07-02
49997   1900-07-02
49998   1900-07-02
49999   1900-07-02
Name: TIMESTAMP, Length: 50000, dtype: datetime64[ns]

In [14]:
asa['DEPARTURE'] = asa['DEPARTURE'].str.replace('-', '/')

In [8]:
asa.dtypes

ID                int64
TIMESTAMP        object
WEBSITE          object
GDS               int64
DEPARTURE        object
ARRIVAL          object
ADULTS            int64
CHILDREN          int64
INFANTS           int64
TRAIN              bool
HAUL_TYPE        object
DISTANCE         object
DEVICE           object
TRIP_TYPE        object
PRODUCT          object
SMS                bool
EXTRA_BAGGAGE      bool
NO_GDS            int64
dtype: object

In [15]:
#gt = pd.to_datetime(asa['ARRIVAL'], format='%d/%B')
gy = pd.to_datetime(asa['DEPARTURE'], format='%d/%B')
#gt
gy

0       1900-07-22
1       1900-07-29
2       1900-07-29
3       1900-07-24
4       1900-08-11
           ...    
49995   1900-07-02
49996   1900-08-11
49997   1900-09-09
49998   1900-07-05
49999   1900-07-06
Name: DEPARTURE, Length: 50000, dtype: datetime64[ns]

In [ ]:
g = sns.pairplot(df_train, vars=['TIMESTAMP', 'WEBSITE', 'GDS', 'DEPARTURE', 'ARRIVAL', 'ADULTS',
       'CHILDREN', 'INFANTS', 'TRAIN', 'HAUL_TYPE', 'DISTANCE', 'DEVICE',
       'TRIP_TYPE', 'PRODUCT', 'SMS', 'NO_GDS'], hue='EXTRA_BAGGAGE')
plt.show(g)

<a id="3.1"></a>
#### Outliers Check

In [ ]:
sns.catplot(data=df_train,kind='box',height=10,aspect=2, palette="GnBu_d");

Let's check if we are facing a balanced dataset:

In [ ]:
df_train['EXTRA_BAGGAGE'].value_counts()

As we can see we are facing a desbalanced dataset. <br>
We will handle this situation further on the following section.

In [ ]:
df_train.columns

In [ ]:
x_train = df_train.drop('EXTRA_BAGGAGE', axis=1)
y_train = df_train.drop(['TIMESTAMP', 'WEBSITE', 'GDS', 'DEPARTURE', 'ARRIVAL', 'ADULTS',
                    'CHILDREN', 'INFANTS', 'TRAIN', 'HAUL_TYPE', 'DISTANCE', 'DEVICE',
                    'TRIP_TYPE', 'PRODUCT', 'SMS', 'NO_GDS'], axis=1)

<a id="5"></a>
### 5. Oversampling

As we have a desbalanced dataset, we will use oversampling in order to balance the classes. <br>

*The technique that we are going tu use it's called SMOTENC (Synthetic Minority Over-sampling Technique for Nominal and Continuous) (https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTENC.html).*

What this technique does is create instances and registers of the class with fewer instance in an "intelligent" way. But taking into account categorical features, unlike SMOTE that will create values such as 0.65 for example when varibles take values of 0 or 1. 

In [ ]:
# We initialize SMOTE
smt = SMOTE()

# We apply oversampling in our train dataset
x_train, y_train = smt.fit_sample(x_train, y_train)

In [ ]:
# Let's check the amount of records in each class
y_train['EXTRA_BAGGAGE'].value_counts()

<a id="4"></a>
### 4. Modeling

## Greetings

I wanted to finish this test by giving thanks for the opportunity to work in such an amazing company that I have been looking up for many years now.<br>
It would be like a dream to be part of such an amazing project as I have been and it would be great to . <br>
I hope I left a good impresion and you take me into account for this position. 

I look forward to hearing from you.

Kind regards,

Tomas Ruan

*__Autor:__ Tomas Ruan Rollan*
<br>
*__LinkedIn:__ https://www.linkedin.com/in/tomas-ruan/*
<br>
*__Email:__ tomruarol@gmail.com*